In [29]:
import pandas as pd
import os

# Load the master dataframe
df_10k = pd.read_csv('../data/askme-qa/raw_data_10k.csv')

# Directory structure
base_dir = '../data/hitl'
annotators = ['HITL_Alex', 'HITL_Atheer', 'HITL_Hend', 'HITL_Xiaoyuan']
file_names = ['Copy of answers_samples.xlsx', 'Copy of questions_samples.xlsx', 'Copy of questions_zs_samples.xlsx']

# Initialize empty DataFrames for each type of samples
df_answers = None
df_questions = None
df_questions_zs = None

# Iterate over each annotator and each file
for annotator in annotators:
    for file_name in file_names:
        file_path = os.path.join(base_dir, annotator, file_name)
        df = pd.read_excel(file_path)

        # Drop "instructions" column and unnamed columns
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        if 'instructions' in df.columns:
            df = df.drop(columns=['instructions'])
        if 'Comments' in df.columns:
            df = df.drop(columns=['Comments'])
        # Drop columns with names that are not in English
        df = df.drop(columns=[col for col in df.columns if not col.isascii()])

        # Rename "annotation" column to include the annotator's name
        if 'annotation' in df.columns:
            annotator_name = annotator.split('_')[-1].lower()
            df = df.rename(columns={'annotation': f'annotation_{annotator_name}'})

        # Merge data to respective DataFrames horizontally
        if 'answers' in file_name:
            if df_answers is None:
                df_answers = df
            else:
                df_answers = df_answers.merge(df, on='answer_id', how='outer', suffixes=('', f'_{annotator_name}'))
        elif 'questions_zs' in file_name:
            if df_questions_zs is None:
                df_questions_zs = df
            else:
                df_questions_zs = df_questions_zs.merge(df, on='question_id', how='outer', suffixes=('', f'_{annotator_name}'))
        else:  # 'questions' in file_name
            if df_questions is None:
                df_questions = df
            else:
                df_questions = df_questions.merge(df, on='question_id', how='outer', suffixes=('', f'_{annotator_name}'))

# Append additional fields from df_10k
df_answers = df_answers.merge(df_10k[['id_answer', 'value']], left_on='answer_id', right_on='id_answer', how='left').drop(columns=['id_answer'])
df_questions_zs = df_questions_zs.merge(df_10k[['question_id', 'is_answerable_zs']], on='question_id', how='left')
df_questions = df_questions.merge(df_10k[['question_id', 'is_answerable_ic']], on='question_id', how='left')

# Rename columns
df_questions = df_questions.rename(columns={'is_answerable_ic': 'annotation_llama3_70b'})
df_questions_zs = df_questions_zs.rename(columns={'is_answerable_zs': 'annotation_llama3_70b'})
df_answers = df_answers.rename(columns={'value': 'annotation_llama3_70b'})

# Replace Y with True and N with False for all annotation columns 
for col in df_questions.columns:
    if col.startswith('annotation_'):
        df_questions[col] = df_questions[col].replace({'Y': True, 'N': False})
for col in df_questions_zs.columns:
    if col.startswith('annotation_'):
        df_questions_zs[col] = df_questions_zs[col].replace({'Y': True, 'N': False})


# Save the updated DataFrames to CSV files
df_questions.to_csv('../data/hitl/questions.csv', index=False)
df_questions_zs.to_csv('../data/hitl/questions_zs.csv', index=False)
df_answers.to_csv('../data/hitl/answers.csv', index=False)


In [30]:
df_questions_zs

,question_id,question,annotation_alex,question_atheer,annotation_atheer,question_hend,annotation_hend,question_xiaoyuan,annotation_xiaoyuan,annotation_llama3_70b
0,2602,Which state is home to Delaware Route 1 Business?,NaN,Which state is home to Delaware Route 1 Business?,True,Which state is home to Delaware Route 1 Business?,True,Which state is home to Delaware Route 1 Business?,True,True
1,2602,Which state is home to Delaware Route 1 Business?,NaN,Which state is home to Delaware Route 1 Business?,True,Which state is home to Delaware Route 1 Business?,True,Which state is home to Delaware Route 1 Business?,True,True
2,2936,"In which township is the first land grant, kno...",True,"In which township is the first land grant, kno...",NaN,"In which township is the first land grant, kno...",True,"In which township is the first land grant, kno...",False,False
3,2936,"In which township is the first land grant, kno...",True,"In which township is the first land grant, kno...",NaN,"In which township is the first land grant, kno...",True,"In which township is the first land grant, kno...",False,False
4,3093,"In what year was ""Shadows in the Desert: Ancie...",True,"In what year was ""Shadows in the Desert: Ancie...",NaN,"In what year was ""Shadows in the Desert: Ancie...",True,"In what year was ""Shadows in the Desert: Ancie...",True,True
...,...,...,...,...,...,...,...,...,...,...
195,38478,Why was IGN particularly looking forward to th...,NaN,Why was IGN particularly looking forward to th...,False,Why was IGN particularly looking forward to th...,True,Why was IGN particularly looking forward to th...,False,False
196,38731,"What type of compound is cucurbitin, which is ...",True,"What type of compound is cucurbitin, which is ...",NaN,"What type of compound is cucurbitin, which is ...",True,"What type of compound is cucurbitin, which is ...",True,True
197,38731,"What type of compound is cucurbitin, which is ...",True,"What type of compound is cucurbitin, which is ...",NaN,"What type of compound is cucurbitin, which is ...",True,"What type of compound is cucurbitin, which is ...",True,True
198,38827,What type of navigational aid were the Mosquit...,True,What type of navigational aid were the Mosquit...,True,What type of navigational aid were the Mosquit...,NaN,What type of navigational aid were the Mosquit...,True,True
